In [1]:
import os
import plotly
import geojson
import numpy as np
import pandas as pd
import networkx as nx
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly import tools as tls
from plotly.offline import iplot, init_notebook_mode
#Imports necessários para a criação do grafo e colocar o grafo no mapa.
#Obs.: Alguns desses são necessários instalar no computador (plotly, mapbox, geojson, os), pip install talvez der certo...


init_notebook_mode(connected=True)
#Comando para que o plotly funcione no modo offline


mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
plotly.tools.set_credentials_file(username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2')
#Comandos Para logar na conta do plotly e usar o mapa disponível do mapbox
#Everlon: username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2'
#Fernando: username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG'




def Casos(arq,semana):
    """
    Lê o arquivo do ano com informação dos casos epidêmicos e semana de ocorrência. 
    Retorna um Dataframe com as informações de ocorrêcias por bairro.
    """

    tabela_oficial = pd.read_csv(
        'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv' ,
        sep=',',
        parse_dates = True, 
        na_values = 0, 
        encoding='latin-1',
        low_memory=False
    ).rename(index=str, columns={"Latitude":"latitude","Longitude":"longitude"})
    
    T=pd.read_csv(arq, 
                  parse_dates = True, 
                  na_values = 0, 
                  encoding='latin-1',
                  low_memory=False)
    
    L=T[T['semana']==semana]
    bairro=list(L['bairro'])
    #data=dict([(i, bairro.count(i)) for i in tabela_oficial['bairro']])
    #Funciona da mesma forma que:
    data={}
    for i in tabela_oficial['bairro'].map(lambda x: x.strip()):
        data[i]=[bairro.count(i)]
    return pd.DataFrame(data)  




def correlation_matrix(arq,t0,t1,e=2,absolute=True):
    """
    Recebe informações de ano(s), semana inicial e final, valor de Euler (por padrão igual a 2) e método 
    (por padrão, absoluto). 
    Se absolute=False, considera-se apenas valores de correlação maior que 0, se 'original', retorna-se as correlações 
    positivas, nulas e negativas.
    """
    
    C=Casos(arq,t0)
    
    for i in range(t0,t1):
        C.loc[i]=Casos(arq,i+1).loc[0]
        
    if absolute=='original':
        matrix=np.array(C.corr().fillna(0))
        filter_matrix=[[0 if matrix[i,j]<1-e else matrix[i,j] for i in range(0,94)] for j in range(0,94)] 
    else:
        if absolute==True:
            matrix=np.absolute(np.array(C.corr().fillna(0)))
            filter_matrix=[[0 if matrix[i,j]<1-e else matrix[i,j] for i in range(0,94)] for j in range(0,94)]
        else:
            matrix=np.array(C.corr().fillna(0))
            M=np.array([[0 if matrix[i,j]<0 else matrix[i,j] for i in range(0,94)] for j in range(0,94)])
            filter_matrix=[[0 if M[i,j]<1-e else M[i,j] for i in range(0,94)] for j in range(0,94)]
  
    return pd.DataFrame(filter_matrix)




def graph_from_matrix(arq,t0,t1,e,absolute=True,p=False):
    """
    Retorna o gráfico a partir da função de correlação correlation_matrix. 
    Por padrão, o gráfico não é desenhado (p=False).
    """
    
    tabela=pd.read_csv(
        'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv' ,
        sep=',',
        parse_dates = True, 
        na_values = 0, 
        encoding='latin-1',
        low_memory=False
    ).rename(index=str, columns={"Latitude":"latitude","Longitude":"longitude"})
    #tabela=pd.read_csv('Tabela_Oficial_2014.csv',na_values=0)
    tabela.fillna(0)
    
    Graph=nx.from_numpy_matrix(np.array(correlation_matrix(arq,t0,t1,e,absolute)))
    
    mapping={i:i+1 for i in range(0,94)}
    H=nx.relabel_nodes(Graph, mapping)
    for i in range(0,94):
        H.add_node(i+1, pos=(tabela['longitude'][i], tabela['latitude'][i]))
        H.node[i+1]['codigo']=tabela['codigo'][i]
        H.node[i+1]['bairro']=tabela['bairro'][i]
        H.node[i+1]['latitude']=tabela['latitude'][i]
        H.node[i+1]['longitude']=tabela['longitude'][i]
    H.nodes(data=True)  
    #Graph=nx.from_numpy_matrix(np.array(correlation_matrix(arq,t0,t1,e)))
    #nx.draw(Graph, nx.get_node_attributes(F, 'pos'), with_labels=True, node_size=200)
    
    if p==True:
        nx.draw(H, nx.get_node_attributes(H, 'pos'), with_labels=True, node_size=200)
        plt.show()
        
    return H
#graph_from_matrix('2015.csv',1,52,0.2,'original',True)




def plotly_graph(arq , titulo, t0 ,t1 , e, plot = False ,absolute = True ,user = 'GoldSeed' ,api = '1cuptk54PTYZG8SnX1W2'):
    """
    Chama o Grafo a partir da função graph_from_matrix.
    Essa função gera um plotly.graph_objs.Figure que têm o Grafo no mapa de recife, onde tem 2 subplots.
    Um mapa com Grafo, e outro mapa com um scatterplot onde o tamanho de cada ponto é de acordo com a quantidade de arestas 
    do Grafo. 
    titulo é o título do mapa, por exemplo: Grafo de 2013 
    Se for dado plot como True, essa função desenha o mapa. (Por padrão, plot = False)
    Lembre-se que você precisará criar uma conta no Plotly e exibir seu login e api_key para esta função.
    Por padrão está conectado na conta gratuíta GoldSeed
    """
   
    G=graph_from_matrix(arq,t0,t1,e,absolute)
    
    mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
    plotly.tools.set_credentials_file(username = user, api_key = api)
    #Everlon: username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2'
    #Fernando: username = 'fansantos', api_key = 'I9HKCsAXlgB1hEMlqcYG'
    
    geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
    #Puxa o arquivo em geojson encontrado no dados Recife que mostra os limites de cada bairro. 
    
    df=pd.read_csv(
        'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv' ,
        sep=',',
        parse_dates = True, 
        na_values = 0, 
        encoding='latin-1',
        low_memory=False).rename(index=str, 
                                 columns={"co_bairro_residencia": "codigo", 
                                          "no_bairro_residencia": "bairro",
                                          "lat":"latitude","long":"longitude"})

    
    Title = titulo + '<br> Semanas de ' + str(t0) + ' a '+str(t1) + '          Parâmetro e = '+str(e)
    #Define como vai ser escrito o título do mapa
    
    
    edge_trace = dict(
        type='scattermapbox',
        lon=[],
        lat=[],
        mode='lines',
        line=dict(width=1.5,color='crimson'), 
        opacity = 0.3,
        hoverinfo='none'
    )
    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['lon'] += tuple([x0, x1, None])
        edge_trace['lat'] += tuple([y0, y1, None])
    #Desenha as arestas do grafo no 1º mapa
        

    node_trace = dict(
        type='scattermapbox',
        lon=[],
        lat=[],
        text=[str(i) for i in df['codigo']],
        mode='markers+text',
        marker=dict(
            opacity = 0.9,
            showscale=True,
            # Opções de colorscales para os vértices e colorbar
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Electric',
            reversescale=True,
            color=[G.degree(i) for i in range(1,len(G.degree)+1)],
            cmin = 0,
            cmax = 94,
            size= 20,
            colorbar=dict(
                tickangle = -90,
                thickness = 25,
                title = 'Quantidade de Arestas',
                titleside = 'right',
                titlefont = dict(size = 20),
                tickfont = dict(size = 16)
            )
        )
    )
    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['lon'] += tuple([x])
        node_trace['lat'] += tuple([y])
    #Desenha os vértices e pinta eles de acordo com a quantidade de arestas no 1º mapa
        
    
    node_size_trace = dict(
        type='scattermapbox',
        subplot='mapbox2',
        lon=[],
        lat=[],
        text=[str(i) for i in df['codigo']],
        mode='markers+text',
        marker=dict(
            opacity = 0.9,
            showscale=False,
            colorscale='Electric',
            reversescale=True,
            color=[G.degree(i) for i in range(1,len(G.degree)+1)],
            cmin = 0,
            cmax = 94,
            size= [G.degree(i) for i in range(1,len(G.degree)+1)], 
        )
    )
    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_size_trace['lon'] += tuple([x])
        node_size_trace['lat'] += tuple([y])
    #Desenha os vértices, pinta eles e altera o tamanho deles de acordo com a quantidade de arestas no 2º mapa
    #Obs.: Nesse segundo mapa não é desenhado o colorbar, pois seria o mesmo do 1º
    
    
    Layout = dict(
            title = Title,
            titlefont = dict(size = 30),
            width = 1350, 
            height = 900,
            autosize = False,
            showlegend=False,                     
            mapbox=dict(
                domain = dict( x = [0,0.495], y=[0,1]),
                layers=[
                    dict(
                        sourcetype = 'geojson',
                        source = geojson,
                        type = 'line',
                        line=dict(width=2),
                        opacity = 0.5,
                        color = 'gray'
                    )
                ],
                accesstoken=mapbox_access_token,
                center=dict(
                    lat=-8.04,
                    lon=-34.937
                ),
                 zoom = 11
            ),
            
            mapbox2=dict(
                domain = dict( x = [0.505,1], y=[0,1]),
                layers=[
                    dict(
                        sourcetype = 'geojson',
                        source = geojson,
                        type = 'line',
                        line=dict(width=2),
                        opacity = 0.5,
                        color = 'gray'
                    )
                ],
                accesstoken=mapbox_access_token,
                center=dict(
                    lat=-8.04,
                    lon=-34.937
                ),
             zoom = 11
            )
    )
    #Dá informações do mapa (como por exemplo o arquivo geojson onde limita os bairros, o zoom, o centro do mapa, etc)         
    #Dá a posição dos subplots, e o tamanho da imagem do mapa.

    
    fig = go.Figure(data=[edge_trace, node_trace, node_size_trace], layout = Layout)     
    #Cria um plotly.graph_objs.Figure com informações da figura (os mapas com o subplots)
    #Outra forma de ver esse comando é como o comando que uni todas as informações usadas anteriormentes

    if plot == True:
        iplot(fig)
    
    return fig

In [2]:
#Exemplo de criar um mapa com as informaçoes vistas antes:

T2013 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Programas/2013.csv'
T2014 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Programas/2014.csv'
T2015 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Programas/2015.csv'
T2016 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Programas/2016.csv'
#Link do Github com os arquivos csv com informações semanais dos casos dos anos 2013, 14, 15 e 16.


fig = plotly_graph(T2013, 'Grafo de 2013', 1 ,52 , 0.55, True)
#Exemplo chamando o Grafo no mapa do ano de 2013 das semanas 1 a 52 com o número de euler = 0.55

#para salvar a imagem no computador pode ser usado o comando do plotly: 
#py.image.save_as(fig, 'nomedoarquivo',format = 'png', scale=1.1)

In [3]:
#Outros mapas: Mapa com tabela dizendo o nome dos estados a partir do código

df=pd.read_csv(
    'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela_de_Bairros.csv',
    sep=',',
    parse_dates = True, 
    na_values = 0, 
    encoding='latin-1',
    low_memory=False
    )
df = df.fillna(0)

mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
plotly.tools.set_credentials_file(username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2')

data_df = go.Table(
    domain = dict(x=[0,0.15], y=[0,1]),
    columnwidth = [25] + [60],
    header = dict(
        values = ['Código','Bairro'],
        height = 25,
        line = dict(color='rgb(50, 50, 50)'),
        font = dict(color=['rgb(45, 45, 45)'] , size=14),
        fill = dict(color='Cyan')
    ),
    cells = dict(
        values = [[str(i) for i in df['codigo']],df['bairro']],
        line = dict(color='#506784'),
        font = dict(color=['rgb(40, 40, 40)'], size=10),
        height = 18,
        fill = dict(color=['lightblue'])
    )
)
#A partir do go.Table, é criado uma tabela com o nome e o código de cada bairro como um subplot.

data_map = go.Scattermapbox(
        mode = 'markers+text',
        lat=df['Latitude'],
        lon=df['Longitude'],
        text = [str(i) for i in df['codigo']],
        marker = dict(
            color = 'white',
            size=20,
            opacity=1
        )
       )
#Scatterplot no mapa com número dos bairros dentro de cada ponto
    

layout = go.Layout(
    title = 'Mapa ', #Título do Mapa
    width = 1600,
    height = 1900,
    mapbox=dict(
        domain = dict(x=[0.16,1], y=[0,1]),
        layers=[
            dict(
                sourcetype = 'geojson',
                source = geojson,
                 type = 'line',
                opacity = 0.8,
                color = 'gray',
                line=dict(width=3)
            )
        ],
        accesstoken=mapbox_access_token,
        center=dict(
             lat=-8.043,
            lon=-34.937
        ),
         zoom = 12.3
    )
)
#informação de subplot e do mapa.


Fig = go.Figure( data = [data_df, data_map], layout = layout)
#Unindo 

iplot(Fig)
#para salvar a imagem no computador pode ser usado o comando do plotly: 
#py.image.save_as(Fig, 'nomedoarquivo',format = 'png', scale=1.1)

In [4]:
#Outros mapas: Mapa de casos (Anual)

C2013 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela2013.csv'
C2014 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela2014.csv'
C2015 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela2015.csv'
C2016 = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Dengue/Dados%20de%20Recife/Tabela2016.csv'
#Link do Github com os arquivos csv com informações anuais dos casos dos anos 2013, 14, 15 e 16.
#Esses links serão usados nesse tópico e no próximo



def MapCasos(arq, tit, plot = False):
    """
    Função onde você dá o link/localização (no caso se tiver o arquivo no computador) do arquivo, o título do mapa e se deseja 
    plotar o mapa (plot = True) ou não (plot = False). 
    Em retorno recebe um plotly.graph_objs.Figure que tem um scatterplot cujo o raio dos pontos é de acordo com a quantidade de
    casos anuais da região.
    """
    
    df = pd.read_csv(arq, 
                 parse_dates = True, 
                 sep = ',', 
                 na_values = 0,
                 encoding='latin-1',
                 low_memory=False)
    df= df.fillna(0)
    
    mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
    geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
    plotly.tools.set_credentials_file(username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2')

    sizes=[]
    for n in round(df['Casos']*70/max(df['Casos']),0):
        if n<9:
            sizes.append(9)
        else:
            sizes.append(n)
            
    data = go.Scattermapbox(
         lat=df['Latitude'],
         lon=df['Longitude'],
         text = [str(i) for i in df['Codigo']],
         mode='markers+text',
         marker = dict(
            showscale=True,
            colorbar = dict(
                tickangle = -90,
                thickness = 25,
                title = 'Número de Casos',
                titleside = 'right',
                titlefont = dict(size = 20),
            ),
            size=sizes,
            color=df['Casos'],
            colorscale='Electric',
            reversescale = True,
            opacity=0.9,
            cauto= False
        )
       )

    

    layout = go.Layout(
        title = tit,
        width = 800, 
        height = 900,
        mapbox=dict(
            layers=[
                dict(
                    sourcetype = 'geojson',
                    source = geojson,
                    type = 'line',
                    opacity = 0.5,
                    color = 'gray'
                )
            ],
            accesstoken=mapbox_access_token,
            center=dict(
                lat=-8.04,
                lon=-34.937
            ),
            zoom = 11
        )
    )
        

    fig = go.Figure( data = [data], layout = layout)
        
    if plot == True:
        iplot(fig)
    
    return(fig)

#Exemplo com os casos de 2015
fig1 = MapCasos(C2015, 'Mapa de Casos de 2015', True)

#para salvar a imagem no computador pode ser usado o comando do plotly: 
#py.image.save_as(fig1, 'nomedoarquivo',format = 'png', scale=1.1)

In [5]:
#Outros mapas: Mapa de casos por população (Anual)

def MapCasosPop(arq, tit, plot = False):
    """
    Função onde você dá o link/localização (no caso se tiver o arquivo no computador) do arquivo, o título do mapa e se deseja 
    plotar o mapa (plot = True) ou não (plot = False). 
    Em retorno recebe um plotly.graph_objs.Figure que tem um scatterplot cujo o raio dos pontos é de acordo com a quantidade de
    casos por população anuais da região.
    """
        
    df = pd.read_csv(arq, 
                 parse_dates = True, 
                 sep = ',', 
                 na_values = 0,
                 encoding='latin-1',
                 low_memory=False)
    df= df.fillna(0)
    
    mapbox_access_token = 'pk.eyJ1Ijoic2lsdmVyc2VlZCIsImEiOiJjam5pNmc3NzAwN3VkM3BsazM3cHRzcmRjIn0.3H4Qchxudkfs8D1PWxLUvA'
    geojson = 'https://raw.githubusercontent.com/AntonioMarcossj/Aedis-Networks/Tabalas-oficiais/Fronteira%20dos%20Bairros.json'
    plotly.tools.set_credentials_file(username = 'GoldSeed', api_key = '1cuptk54PTYZG8SnX1W2')

    sizes=[]
    for n in round(df['Casos/Pop']*70/max(df['Casos/Pop']),0):
        if n<9:
            sizes.append(9)
        else:
            sizes.append(n)
            
            
    data = go.Scattermapbox(
        lat=df['Latitude'],
        lon=df['Longitude'],
        text = [str(i) for i in df['Codigo']],
        mode='markers+text',
        marker = dict(
            showscale=True,
            colorbar = dict(
                tickangle = -90,
                thickness = 25,
                title = 'Número de Casos/Pop',
                titleside = 'right',
                titlefont = dict(size = 20),
            ),
            size=sizes,
            color=df['Casos/Pop'],
            colorscale='Electric',
            reversescale = True,
            opacity=0.9,
            cauto= False
        )
       )

    

    layout = go.Layout(
        title = tit,
        width = 800, 
        height = 900,
        mapbox=dict(
            layers=[
            dict(
                sourcetype = 'geojson',
                source = geojson,
                type = 'line',
                opacity = 0.5,
                color = 'gray'
            )
        ],
        accesstoken=mapbox_access_token,
        center=dict(
              lat=-8.04,
            lon=-34.937
        ),
         zoom = 11
    )
    )
        

    fig = go.Figure( data = [data], layout = layout)
        
    if plot == True:
        iplot(fig)
    
    return(fig)

#Exemplo com os casos/população de 2015
fig2 = MapCasosPop(C2015, 'Mapa de Casos/População de 2015', True)

#para salvar a imagem no computador pode ser usado o comando do plotly: 
#py.image.save_as(fig2, 'nomedoarquivo',format = 'png', scale=1.1)